<a href="https://colab.research.google.com/github/wayne-xyz/GoogleEarthEngineTask/blob/main/Gee_Export_tif_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exporting tif file from GEE Nicif and sentinel

- Nicfi
 - Monthly: [planet.com](https://developers.planet.com/docs/integrations/gee/nicfi/faq/) Analytic Monthly series Basemaps are stored as images in the Image Collections representing a single month in time.  Running a date filter for a specific time period within this month, will not return any results (for example, between 2021-08-02 and 2021-08-19).
 - Monthly update on GEE



## 1.Shared config before excute the exporting

After Excute this part , excute the nicfi or sentinel export:
- Setting the parameters for exporting
- Run the exporting
- Test function of exporting

### Config Setting


In [ ]:
# Google Drive folder name
from google.colab import drive
import time

drive.mount('/content/drive')

# export function only support the root's folder name , not the path
sentinel_folder_name="sentinel_tif_2024Jun"
nicfi_folder_name="nicfi_tif_2024Jun"

# load the exsiting table of all the shapefile
shapefile_table_localpath="/content/drive/My Drive/Dumpsite_Images/shapefile_data.csv"

# minimun shape size of dumpsite
shapefile_size=0.1

# time range
start_date='2023-07-01'
end_date='2024-07-01'

# image zoom



print("Current Time:",time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current Time: 2024-07-25 17:54:55


### Initial EE

In [ ]:
# pre-setting

# Trigger the authentication flow.
import ee
import time

ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-qinheyi')
print("Current Time:",time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))


Current Time: 2024-07-25 17:54:58


### Define exporting function

In [ ]:

# changable scalled for each shape
def export_tif_image(ind,feature,image,date_str='YYYYMM',source_name="Source",folder_name="tif_file"):
    """
    Export the tif file image from GEE (both nicfi and sentinel) to Google Drive

    Args:
        ind(int): index for the shapefile
        feature(feature):shape of certain place
        date_str(str): image's date. format YYYMMDD  or YYYYMM
        source_name(str): source name.
        folder_name(str): folder name.
    """
    # to_export=image.clip(feature.geometry())
    # export_id=feature.get('Index').getInfo()
    # print(image.bandNames().getInfo())
    task=ee.batch.Export.image.toDrive(
        image=image,
        description=f"export_{ind}_{date_str}",
        folder=folder_name,
        region=feature.geometry(),
        scale=10,
        crs='EPSG:4326',
        maxPixels=1e13,
        fileNamePrefix=f"{ind}-{date_str}-{source_name}"
    )
    task.start()
    print(f"Export task start:{ind} ",date_str)

print("Current Time:",time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))


# based on the shape size , the tif will have 2 option , <4ha tif will be 5 , > 4 the tif will be double size of the shape
def export_tif_image_fixedsize(ind,feature,image,date_str='YYYYMM',source_name="Source",folder_name="tif_file",shape_size=1):
    """
        Export the tif file image from GEE (both nicfi and sentinel) to Google Drive

        Args:
            ind(int): index for the shapefile
            feature(feature): shape of certain place
            image: Image to export
            date_str(str): image's date. format YYYYMMDD or YYYYMM
            source_name(str): source name.
            folder_name(str): folder name.
            shape_size(int): size of the shape ha, default 1
        """
        # Convert hectares to square meters (1 hectare = 10,000 square meters).
    if shape_size < 4:
        exportSizeSqMeters = 5 * 10000
    else:
        exportSizeSqMeters = shape_size * 10000 *2


    # Get the centroid of the feature.
    centroid = feature.geometry().centroid()

    # Create a bounding box of 5 hectares centered on the centroid.
    halfSideLength = (exportSizeSqMeters ** 0.5)
    exportRegion = centroid.buffer(halfSideLength).bounds()

    # Start the export task.
    task = ee.batch.Export.image.toDrive(
        image=image.clip(exportRegion),
        description=f"export_{ind}_{date_str}",
        folder=folder_name,
        region=exportRegion,
        scale=10,
        crs='EPSG:4326',
        maxPixels=1e13,
        fileNamePrefix=f"{ind}-{date_str}-{source_name}"
    )
    task.start()
    print(f"Export task started for index {ind} with date {date_str}")

print("Current Time:", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

Current Time: 2024-07-25 17:54:58
Current Time: 2024-07-25 17:54:58


### Load the shape file and filter by size of shape and load the table in LOCAL

In [ ]:
# load the shape file check the shapefile
import pandas as pd

shapefile_table=ee.FeatureCollection('projects/ee-qinheyi/assets/1823_ADRSM')
print('Count of table item:',shapefile_table.size().getInfo())

# filter function 1 based on the size of the shape

size_field_name='AREA_HA'
size_filter=ee.Filter.greaterThan(size_field_name,shapefile_size)
filtered_shape=shapefile_table.filter(size_filter)
print("size of filtering", size_filter.getInfo())
export_shape_count=filtered_shape.size().getInfo()
print('Count of table item after filtering',filtered_shape.size().getInfo())

# get the index list of filtered shapes
filtered_index_list=filtered_shape.aggregate_array('Index').getInfo()

# load the table of the shapefile
shapefile_df=pd.read_csv(shapefile_table_localpath)
print(shapefile_df.head())

Count of table item: 1823
size of filtering {'type': 'Filter.lt', 'leftValue': 0.1, 'rightField': 'AREA_HA'}
Count of table item after filtering 1206
           system:index ACT_SEGREG  AREA_HA  AREA_M2 ARROJO_RR1 ARROJO_RR_  \
0  000000000000000006ad         No  4.00000  40000.0         No         No   
1  000000000000000006af         No  0.50000   5000.0         No         Si   
2  000000000000000006aa         No  0.76505   7650.5         No         No   
3  000000000000000006a8         No  0.17000   1761.0         No         Si   
4  000000000000000006b2         No  0.02000    200.0         No         No   

  CANT_RRSS CATEGORIA CRIANZA_AN                          DEN_BOT  ...  \
0      0.15     NUEVO         No                Botadero Tanccama  ...   
1     0.133     NUEVO         No              Botadero Yura Yurac  ...   
2    0.0639     NUEVO         No  Botadero Huachaypampa-Llipllina  ...   
3     0.085     NUEVO         No                    Botadero Acco  ...   
4     0.066

### Dealing with the limitaion of GEE tasks queue

In [ ]:
# check the task list
max_runing_tasks=2500
task_check_interval = 600  # Time to wait (in seconds) between task checks, sleep 10 mins to check next loop

def check_task_status():
    tasks = ee.batch.Task.list()
    running_tasks = [task for task in tasks if task.state in ['READY', 'RUNNING']]
    return len(running_tasks)

def check_complete_task():
    tasks = ee.batch.Task.list()
    completed_tasks = [task for task in tasks if task.state in ['COMPLETED']]
    return len(completed_tasks)

## 2.GEE Export the tif from nicfi source





### Load nicfi and the date_list

In [ ]:
# Load the nicfi and the check the newest time of image

nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

def getDateRange(imageCollection:ee.ImageCollection):
    dateRange=imageCollection.aggregate_array('system:time_end')
    date_list=[ee.Date(date).format().getInfo() for date in dateRange.getInfo()]
    return date_list

nicfi_date_list=getDateRange(nicfi)
print("Leng",len(nicfi_date_list),nicfi_date_list)

Leng 56 ['2016-06-01T00:00:00', '2016-12-01T00:00:00', '2017-06-01T00:00:00', '2017-12-01T00:00:00', '2018-06-01T00:00:00', '2018-12-01T00:00:00', '2019-06-01T00:00:00', '2019-12-01T00:00:00', '2020-06-01T00:00:00', '2020-09-01T00:00:00', '2020-10-01T00:00:00', '2020-11-01T00:00:00', '2020-12-01T00:00:00', '2021-01-01T00:00:00', '2021-02-01T00:00:00', '2021-03-01T00:00:00', '2021-04-01T00:00:00', '2021-05-01T00:00:00', '2021-06-01T00:00:00', '2021-07-01T00:00:00', '2021-08-01T00:00:00', '2021-09-01T00:00:00', '2021-10-01T00:00:00', '2021-11-01T00:00:00', '2021-12-01T00:00:00', '2022-01-01T00:00:00', '2022-02-01T00:00:00', '2022-03-01T00:00:00', '2022-04-01T00:00:00', '2022-05-01T00:00:00', '2022-06-01T00:00:00', '2022-07-01T00:00:00', '2022-08-01T00:00:00', '2022-09-01T00:00:00', '2022-10-01T00:00:00', '2022-11-01T00:00:00', '2022-12-01T00:00:00', '2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00', '2023-04-01T00:00:00', '2023-05-01T00:00:00', '2023-06-01T00:00:00', '2

In [ ]:


# default image
nicfi_collection_image=nicfi.filterDate(nicfi_date_list[0],nicfi_date_list[-1]).sort('system:time_start', False).first()
print(ee.Date( nicfi_collection_image.date()).format().getInfo())

# print the bands information
print(nicfi_collection_image.bandNames().getInfo())




2024-06-01T00:00:00
['B', 'G', 'R', 'N']


### Execute the export

In [ ]:
# iterate the time list of the nicfi_date_list
# and iterate the shape

# count of each image
count_per_loc=1  # means the lastest one month's data

# index_list
index_list=filtered_index_list


#pending for fix the invoke of the exporting
processing_count=0
nicfi_tif_amount=count_per_loc*len(index_list) # show the amount of tif file
for t in range(len(nicfi_date_list)-1,len(nicfi_date_list)-count_per_loc-1,-1):
    print("Processing:",processing_count,nicfi_date_list[t-1][0:7],"--",nicfi_date_list[t][0:7])
    export_nicfi_basemap=nicfi.filterDate(nicfi_date_list[t-1],nicfi_date_list[t]).sort('system:time_start', False).first()
    for index in index_list:
        processing_count+=1
        select_shape=shapefile_table.filter(ee.Filter.eq('Index',index))
        # get AERA_HA from the shapefile_df
        shape_size=shapefile_df[shapefile_df['Index'] == index]['AREA_HA'].values[0]
        # Test the shape_size and show the procssing_count
        print(f"index:{index}, shape_size:{shape_size}, processing_count:{processing_count} / {nicfi_tif_amount}")

        if processing_count%3==0 and processing_count>2400:
          while check_task_status() >= max_runing_tasks:
              print(f"Waiting for tasks to complete...runing tasks:{check_task_status()}")
              time.sleep(task_check_interval)

        export_tif_image_fixedsize(index,select_shape.first(),export_nicfi_basemap,nicfi_date_list[t][0:7],"nicfi",nicfi_folder_name,shape_size=shape_size)
        # TODO export_nicfi_image(index,select_shape.first(),export_nicfi_basemap,nicfi_date_list[t][0:7] )  pending





Processing: 0 2024-06 -- 2024-07
index:1709, shape_size:4.0, processing_count:1 / 1206
Export task started for index 1709 with date 2024-07
index:1711, shape_size:0.5, processing_count:2 / 1206
Export task started for index 1711 with date 2024-07
index:1706, shape_size:0.76505, processing_count:3 / 1206
Export task started for index 1706 with date 2024-07
index:1704, shape_size:0.17, processing_count:4 / 1206
Export task started for index 1704 with date 2024-07
index:1719, shape_size:1.0, processing_count:5 / 1206
Export task started for index 1719 with date 2024-07
index:1718, shape_size:3.7, processing_count:6 / 1206
Export task started for index 1718 with date 2024-07
index:1743, shape_size:0.8, processing_count:7 / 1206
Export task started for index 1743 with date 2024-07
index:1555, shape_size:0.40864448, processing_count:8 / 1206
Export task started for index 1555 with date 2024-07
index:1707, shape_size:0.23, processing_count:9 / 1206
Export task started for index 1707 with date

###  Export one image TEST

In [ ]:
# filter function 2 based on certain index do the selction
# this function is for test , only selection one location

# RUN this after load the shapefile
test_field_name='Index'
test_field_value=1740
test_select=ee.Filter.eq(test_field_name,test_field_value)
test_select_shape=shapefile_table.filter(test_select)
print("index",test_select_shape.first().get('Index').getInfo())



print('Count of table item:',test_select_shape.size().getInfo())
# TODO export_nicfi_image(test_select_shape.first()) pending
export_tif_image_fixedsize(test_field_value,test_select_shape.first(),export_nicfi_basemap,nicfi_date_list[t][0:7],"nicfi",f"nicif_{test_field_value}",shape_size=25)
export_tif_image(test_field_value,test_select_shape.first(),export_nicfi_basemap,nicfi_date_list[t][0:7],"nicfi",f"nicif_{test_field_value}")


# get the geometry of this selected shape
test_geometry=test_select_shape.geometry()
print(test_geometry.getInfo())
print(test_geometry.area().getInfo())

index 1740
Count of table item: 1
Export task started for index 1740 with date 2024-07
Export task start:1740  2024-07
{'type': 'Polygon', 'coordinates': [[[-80.84707833151006, -5.746746782807356], [-80.84676616394673, -5.747246197879137], [-80.8464050075316, -5.746933993167123], [-80.84625339905642, -5.746679824048988], [-80.84584316788882, -5.7466753836964575], [-80.84558006814474, -5.746720028631912], [-80.84563800928616, -5.746960774541778], [-80.84568260535481, -5.746974125001984], [-80.84597687121577, -5.747009839682905], [-80.84583421491011, -5.747228295317482], [-80.84551317394687, -5.747433477682885], [-80.84525902616313, -5.747384421024927], [-80.84499144796432, -5.747736673204591], [-80.84482201509482, -5.747446853788952], [-80.8446123773197, -5.747558310619363], [-80.844434074977, -5.7476430150083075], [-80.8441977456328, -5.74733980539183], [-80.84382762443725, -5.747281813830131], [-80.8436760110852, -5.747433455974853], [-80.8438632683146, -5.747750022181812], [-80.84347

## 3.GEE Export the tif from sentinel

Usuage description:
- Set the parameter
- Load data
- Excute the Exporting

Support exporting 3 time slot of a month like 1/1-1/10 1/11-1/20 1/20-1/30

- Dealing with the limitation of the exporting. The googld earth engin exporting tasks queue only has 3000 capacity.

`EEException: Too many tasks already in the queue (3000). Please wait for some of them to complete.`

In [ ]:

# bands infor  https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands
# B2 B3 B4 : B G R
# B8: NIR
# B11 : SWIR 1  short wave infrared wavelength: 1613.7, 1610.4
# B12 : SWIR 2  .... wavelentgh 2002.4 2185.7

### Load sentinel data source

In [ ]:
# load sentinel dataset ready


s2_dataset=ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

one_image=ee.Image( s2_dataset.filterDate(start_date,end_date).first())
print(one_image.bandNames().getInfo())
print(one_image.date().format("YYYY-MM-dd").getInfo())


['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']
2023-07-01


### Generate the date list for last year

In [ ]:
# Date list ready

from datetime import datetime, timedelta

# define function to get the date list for 3 time slot a month.
# creat a date list which contains the 3 date of a month and within the data_start to date_end
def generate_dates(start_date, end_date):
    # Convert string dates to datetime objects
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    # Generate a list to store the dates
    dates = []

    # Loop through each month from start to end
    current = start
    while current <= end:
        # Append the 1st, 10th, and 20th of each month to the list
        for day in [1, 10, 20]:
            date = datetime(current.year, current.month, day)
            if date <= end:
                dates.append(date.strftime("%Y-%m-%d"))

        # Move to the next month
        if current.month == 12:
            current = datetime(current.year + 1, 1, 1)
        else:
            current = datetime(current.year, current.month + 1, 1)

    return dates


export_dates=generate_dates(start_date,end_date)
print(export_dates)
print(len(export_dates))

['2023-07-01', '2023-07-10', '2023-07-20', '2023-08-01', '2023-08-10', '2023-08-20', '2023-09-01', '2023-09-10', '2023-09-20', '2023-10-01', '2023-10-10', '2023-10-20', '2023-11-01', '2023-11-10', '2023-11-20', '2023-12-01', '2023-12-10', '2023-12-20', '2024-01-01', '2024-01-10', '2024-01-20', '2024-02-01', '2024-02-10', '2024-02-20', '2024-03-01', '2024-03-10', '2024-03-20', '2024-04-01', '2024-04-10', '2024-04-20', '2024-05-01', '2024-05-10', '2024-05-20', '2024-06-01', '2024-06-10', '2024-06-20', '2024-07-01']
37


#### Define the export date range of the list

In [ ]:

# Choose 2024-06-01  to 2024-07-01
end_in=len(export_dates)
stat_in=-4

print(export_dates[stat_in:end_in])


['2024-06-01', '2024-06-10', '2024-06-20', '2024-07-01']


### Execute the export

In [ ]:
# iterate the time slot for the pair of slot start and end

# dates_list , the dates list accturaly exporting
# filtered_shape : based on the filterd_shape
# filtered_index_list : index list
# shapefile_df : the shapefile dataframe


import sys
import time
time_index=0
export_count=0
dates_list=export_dates[stat_in:end_in]
total_count=len(filtered_index_list)*len(dates_list-1) # 3(date) * 1206(dumpsite count) = 3,618



export_shape_table=filtered_shape.sort('Index',False)
for i in range(len(dates_list)-1):
    time_index+=1
    print( dates_list[i],dates_list[i+1])
    for index in filtered_index_list:
        select_shape=export_shape_table.filter(ee.Filter.eq('Index',index))
        # get AERA_HA from the shapefile_df
        shape_size=shapefile_df[shapefile_df['Index'] == index]['AREA_HA'].values[0]
        # Test the shape_size
        print(f"index:{index}, shape_size:{shape_size}, date{dates_list[i]}")

        export_s2_image=s2_dataset.filterDate(dates_list[i],dates_list[i+1]).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',90)).mean()  # only use mean rather than first()

        # check the tasks queue running tasks and max runing tasks
        # only the export_count even then check the tasks queue and delay it
        if export_count%3==0 and export_count>2000:
          while check_task_status() >= max_runing_tasks:
              print(f"Waiting for tasks to complete...runing tasks:{check_task_status()}")
              time.sleep(task_check_interval)

        # export function
        # export_tif_image(index,select_shape.first(),export_s2_image,export_dates[i].replace('-',''),"sentinel",sentinel_folder_name)
        export_tif_image_fixedsize(index,select_shape.first(),export_s2_image,dates_list[i].replace('-',''),"sentinel",sentinel_folder_name,shape_size=shape_size)
        export_count+=1
        print(f"exporting task submission progress:{export_count} / {total_count}")



Streaming output truncated to the last 5000 lines.
Export task started for index 1139 with date 20240610
exporting task submission progress:1954 / 4824
index:1140, shape_size:0.19928219, date2024-06-10
Export task started for index 1140 with date 20240610
exporting task submission progress:1955 / 4824
index:1339, shape_size:2.90935467, date2024-06-10
Export task started for index 1339 with date 20240610
exporting task submission progress:1956 / 4824
index:291, shape_size:0.11446934, date2024-06-10
Export task started for index 291 with date 20240610
exporting task submission progress:1957 / 4824
index:292, shape_size:0.4629134, date2024-06-10
Export task started for index 292 with date 20240610
exporting task submission progress:1958 / 4824
index:421, shape_size:0.29851771, date2024-06-10
Export task started for index 421 with date 20240610
exporting task submission progress:1959 / 4824
index:948, shape_size:0.26797022, date2024-06-10
Export task started for index 948 with date 2024061

### Check the exporting tasks queue

In [ ]:

tasks = ee.batch.Task.list()
print(check_task_status())
print(check_complete_task())

0
17420


In [ ]:
# check the exporting status
ee.data.listOperations()

### Cancel the tasks in the queue

In [ ]:
# Function to cancel all tasks
def cancel_all_tasks():
    task_list = ee.batch.Task.list()
    canceled_count=0
    for task in task_list:
        task_id = task.status()['id']
        task_name = task.status()['description']
        task_state = task.status()['state']
        if task_state in ['READY', 'RUNNING','PENDING']:
            print(f'Cancelling task: {task_name} (ID: {task_id}),canceled_count:{canceled_count}')
            task.cancel()
            canceled_count+=1
        else:
            print(f'Skipping task: {task_name} (ID: {task_id}) - State: {task_state}')

# Execute the function to cancel all tasks
cancel_all_tasks()

### Test one image from the sentinel



In [ ]:
# get one shape from shapefile
selected_index=699
test_feature_sentinel=shapefile_table.filter(ee.Filter.eq('Index',selected_index)).first()
print(test_feature_sentinel.getInfo())

# get the image of the shape and time range
test_sentinel_image=s2_dataset.filterDate("2024-04-01","2024-04-30" ).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).mean()  # only use mean rather than first()
print(test_sentinel_image.bandTypes().getInfo())

## export the sentinel image in two size

export_tif_image(selected_index,test_feature_sentinel,test_sentinel_image,"2024-04-01".replace('-',''),"sentinel",f"{selected_index}sentinel_flexible")
export_tif_image_fixedsize(selected_index,test_feature_sentinel,test_sentinel_image,"2024-04-01".replace('-',''),"sentinel",f"{selected_index}sentinel_fixed",shape_size=25)

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-78.27081487076823, -9.138958826400813], [-78.27081035244588, -9.138998919382473], [-78.27078359007099, -9.13899448037562], [-78.27073458362648, -9.138981103188492], [-78.2706988714689, -9.138967743991941], [-78.27066324619004, -9.138954387625395], [-78.27063196544736, -9.13894540413848], [-78.2706007715863, -9.138932026114945], [-78.27056062794081, -9.138918688579643], [-78.2705338655019, -9.138914249565362], [-78.27051605283584, -9.138909770064448], [-78.2704892903855, -9.138900850717059], [-78.27047147771182, -9.138891973854436], [-78.27037780906564, -9.138838395993961], [-78.27028865868053, -9.13877152431966], [-78.27021732093937, -9.138718009869013], [-78.27015484606875, -9.13865117715118], [-78.27011478914316, -9.138597602668234], [-78.27008350823614, -9.138557505974076], [-78.27005231421118, -9.138508451528253], [-78.27003893292743, -9.13845939629249], [-78.27002112017692, -9.138392524531808], [-78.27001217035

###

[{'name': 'projects/ee-qinheyi/operations/E2K4MVEWAOCPBRYY6ATEJESE',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'PENDING',
   'description': 'export_568',
   'priority': 100,
   'createTime': '2024-07-25T16:40:08.593300Z',
   'updateTime': '2024-07-25T16:40:08.593300Z',
   'startTime': '1970-01-01T00:00:00Z',
   'type': 'EXPORT_IMAGE'}},
 {'name': 'projects/ee-qinheyi/operations/SWWLIHYTNVLTPF36J3MBGX6P',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'PENDING',
   'description': 'export_543',
   'priority': 100,
   'createTime': '2024-07-25T16:40:08.432728Z',
   'updateTime': '2024-07-25T16:40:08.432728Z',
   'startTime': '1970-01-01T00:00:00Z',
   'type': 'EXPORT_IMAGE'}},
 {'name': 'projects/ee-qinheyi/operations/O4N3WI5QQ7EDYYCOIIN6M6NU',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'PENDING',
   'descriptio

In [ ]:
# test showing it

import geemap
map=geemap.Map()
vis_params = {
    'min': 0,
    'max': 3000,  # why the simple is 0.3
    'bands': ['B4', 'B3', 'B2']
}

map.setCenter(83.277, 17.7009, 12);
map.addLayer(test_sentinel_image, vis_params, 'Sentinel-2')

